In [ ]:
#Required libraries
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
import sys
import os
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN, MSO_ANCHOR
from pptx.dml.color import RGBColor
from pd2ppt import df_to_powerpoint, df_to_table
from pd2ppt import df_to_powerpoint, df_to_table
from pptx.chart.data import ChartData
from pptx.enum.chart import XL_CHART_TYPE
from pptx.enum.chart import XL_LABEL_POSITION
from pptx.enum.chart import XL_LEGEND_POSITION
from scipy.stats import pearsonr

In [ ]:
#Functions to create base tables for slides

#Creates demographic tables to produce charts
def demographic_table(group_data, topic):    
    #Add 'Missing' as a category
    group_data[topic].fillna('Missing', inplace= True)
    #Generate table
    chart = pd.crosstab(index=group_data[topic], columns=group_data['encounter_type'], colnames=[''], dropna=False,  normalize=1).rename(columns={"IP": "Inpatient", "OP": "Outpatient"})
    chart = round(chart,5)
    chart = pd.DataFrame(chart)
    #Some categories have a count of 0 - these need to be represented in the charts. The following code adds these categories and sets the order
    chart['Order'] = np.zeros(len(chart.index))
    wb = pd.ExcelFile('data_ref.xlsx')
    ref = wb.parse(topic)
    #Set Missing as the last category
    chart.loc[chart.index == "Missing", "Order"] = len(chart.index)+1
    for i in ref['Id']:
        if i in chart.index:
            #set the order of existing categories
            chart.loc[chart.index == i, "Order"] = int(ref[ref["Id"]==i]["Order"])
        else:
            #add unrepresented categories with count of 0 
            df = pd.DataFrame([[0,0, int(ref[ref["Id"]==i]["Order"])]],index=[i],columns=["Inpatient", "Outpatient", "Order"])
            chart = chart.append(df)
    chart = chart.sort_values(by=["Order"])        
    chart = chart.drop(['Order'], axis=1)
    #Rename indexes to reflect group labels in question dictionary
    change_label = {}
    for i in chart.index:
        if i == "Missing":
            change_label[i] = i
        else:
            change_label[i] = ref[ref['Id']==i]["Desc"].values[0]
        updated_chart = chart.rename(index=change_label)
    return updated_chart 

In [ ]:
#Creates top and bottom tables
def tb_table(group_data, group, q_index, report_group):    
    if report_group == 'Non-Registered Family':
        tb_data = group_data
    else:
        tb_data = group_data[group_data['encounter_type']==group]
    #Get population size
    n = tb_data.shape[0]
    #setting up table
    items = []
    pos_resp = []
    pos_resp_percent = []
    denom = []
    missing_num = []
    missing_percent = []
    missing_percent_format = []
    #Iteratively gathering data values
    for i in q_index['question_no']:
        #Question content
        items.append(q_index.loc[q_index['question_no']==i,'content'].iloc[0])
        #Number of Positive responses
        pos = tb_data[i][tb_data[i]>=3].value_counts().sum()
        pos_resp.append(pos)
        #Number of missing data
        missing = tb_data[i].isnull().sum()
        missing_num.append(missing)
        denom_tmp = n-missing
        denom.append("n = {0}".format(denom_tmp))
        #Percentages
        pos_resp_percent.append(round((pos/denom_tmp)*100,5))
        missing_percent.append(round((missing/n)*100,5))
        missing_percent_format.append("{0}%".format(round((missing/n)*100,1)))
    #Add data to final table with correct labels
    labels = ['Items', 'Positive Responses (Of those applicable)','Pos Num','Denominator','N/A & Missing (Of overall)','Miss Num' ]
    tb_df = pd.DataFrame(columns=labels)
    tb_df['Items'] = items
    tb_df['Positive Responses (Of those applicable)'] = pos_resp_percent
    tb_df['Pos Num'] = pos_resp
    tb_df['Denominator'] = denom
    tb_df['N/A & Missing (Of overall)'] = missing_percent_format
    tb_df['Miss Num'] = missing_num
    #Determine questions with % of missing data > 10% for future slide
    miss = []
    for x in range(0, tb_df.shape[0]):
        if missing_percent[x] > 10:
            item_miss = q_index[q_index['content']==tb_df['Items'][x]]['question_no'].values[0]
            miss.append("{0}".format(item_miss))
    #Inpatient group fine to proceed and sort data
    if group == "IP":
        #IP Top chart
        tb_df.sort_values(by=['Positive Responses (Of those applicable)'], inplace=True, ascending=False)
        tb_df['Positive Responses (Of those applicable)']
        if report_group == 'Non-Registered Family':
            top_chart = tb_df.head(5)
        else:
            top_chart=tb_df.head(10)
        tmp = ["{0}%".format(round(x,1)) for x in top_chart['Positive Responses (Of those applicable)']]
        top_chart['Positive Responses (Of those applicable)'] = tmp
        #IP Bottom chart
        tb_df.sort_values(by=['Positive Responses (Of those applicable)'], inplace=True)
        if report_group == 'Non-Registered Family':
            bottom_chart = tb_df.head(5)
        else:
            bottom_chart=tb_df.head(10)
        tmp = ["{0}%".format(round(x,1)) for x in bottom_chart['Positive Responses (Of those applicable)']]
        bottom_chart['Positive Responses (Of those applicable)'] = tmp
    else:
        #Outpatient group needs to be filtered for 'Residential or Inpatient' Questions
        rp = "Residential or Inpatient"
        ip_q = q_index[q_index["area"] == rp]['content']
        exclude = pd.DataFrame({'Items':[x for x in ip_q]})
        tb_op_df = tb_df[~tb_df.Items.isin(exclude.Items)]
        #Determine questions with % of missing data > 10% for future slide
        miss = []
        for x in range(0,tb_df.shape[0]):
            if ~tb_df.Items.isin(exclude.Items)[x] == True:
                if missing_percent[x] > 10:
                    item_miss = q_index[q_index['content']==tb_df['Items'][x]]['question_no'].values[0]
                    miss.append("{0}".format(item_miss))
            else:
                continue
        tb_op_df.sort_values(by=['Positive Responses (Of those applicable)'], inplace=True, ascending=False)
        if report_group == 'Non-Registered Family':
            top_chart = tb_op_df.head(5)
        else:
            top_chart=tb_op_df.head(10)
        tmp = ["{0}%".format(round(x,1)) for x in top_chart['Positive Responses (Of those applicable)']]
        top_chart['Positive Responses (Of those applicable)'] = tmp
        tb_op_df.sort_values(by=['Positive Responses (Of those applicable)'], inplace=True)
        if report_group == 'Non-Registered Family':
            bottom_chart = tb_op_df.head(5)
        else:
            bottom_chart=tb_op_df.head(10)
        tmp = ["{0}%".format(round(x,1)) for x in bottom_chart['Positive Responses (Of those applicable)']]
        bottom_chart['Positive Responses (Of those applicable)'] = tmp
    return top_chart, bottom_chart, n, miss

In [ ]:
#Creates overall tables
def overall_tables(group_data, report_group, q_index, q_overall, group):
    overall_q = q_overall['question_no']
    #Setting up tables
    overall_data = group_data[[x for x in overall_q]]
    overall_data['encounter_type'] = group_data['encounter_type']
    if (report_group == 'Non-Registered Family'):
        group = ['(All Programs)']
    chart_34 = pd.DataFrame({'Type': group})
    chart_4 = pd.DataFrame({'Type': group})
    for j in overall_q:
        #Counts number each rank appears from 1 to 4
        if (report_group == 'Non-Registered Family'):
            df = overall_data.groupby(j).size()
        else:
            df = overall_data.groupby(['encounter_type', j]).size()
        count_34 = []
        count_4 = []
        for i in chart_34['Type']:
            if (report_group == 'Non-Registered Family'):
                #calculate denominator
                n = df.sum()
                #generate data for strongly agree and agree table
                count_34.append(round((sum(df[df.index > 2])/n),3))
                #generate data for strongly agree table
                count_4.append(round((sum(df[df.index > 3])/n),3))
            else:
                #calculate denominator
                n = df[i].sum()
                #generate data for strongly agree and agree table
                count_34.append(round((sum(df[i][df[i].index > 2])/n),3))
                #generate data for strongly agree table
                count_4.append(round((sum(df[i][df[i].index > 3])/n),3))
        #Add data to final table with correct labels
        chart_34[q_overall.loc[q_overall['question_no']==j,'content'].iloc[0]] = count_34
        chart_4[q_overall.loc[q_overall['question_no']==j,'content'].iloc[0]] = count_4
    chart_34 = chart_34.set_index('Type')
    chart_4 = chart_4.set_index('Type')
    try:
        chart_34 = chart_34.rename(index={'IP': 'Inpatient', 'OP':'Outpatient'})
        chart_4 = chart_4.rename(index={'IP': 'Inpatient', 'OP':'Outpatient'})
    except:
        chart_34 = chart_34.rename(index={'OP':'Outpatient'})
        chart_4 = chart_4.rename(index={'OP':'Outpatient'})
    return chart_34, chart_4

In [ ]:
#Creates correlation tables to produce charts
def corr_tables(group_data, group, q_index, q):
    if report_group == 'Non-Registered Family':
        corr_data = group_data
    else:
        corr_data = group_data[group_data['encounter_type']==group]
    items = []
    r_values = []
    n_values = []
    for i in q_index['question_no'] :
        #Set up table to calculate correlation coefficient (pearson)
        q_concat = pd.concat([corr_data[q], corr_data[i]], axis=1,sort=False)
        n = len(q_concat.dropna())
        tmp = q_concat.dropna()
        #calculate correlation coefficent and p-value
        try:
            r,p = pearsonr(tmp[q], tmp[i])
        except ValueError:
            continue
        corr = round(r,3)
        #accuracy check - uncomment below
        #print(q, n, i, corr, '{0:f}'.format(p))
        #filter for questions with p-value under 0.01
        if p <= 0.0001: 
            items.append(q_index.loc[q_index['question_no']==i,'content'].iloc[0])
            r_values.append(corr)
            n_values.append(n)
        else:
            continue
    #Add data to final table with correct labels
    labels = ['Items', 'Correlation Coefficient (r)', 'n']
    corr_df = pd.DataFrame(columns=labels)
    corr_df['Items'] = items
    corr_df['Correlation Coefficient (r)'] = r_values
    corr_df['n'] = n_values
    #If Inpatient continue to sort
    if group == "IP":
        corr_df.sort_values(by=['Correlation Coefficient (r)'], inplace=True, ascending=False)
        top_chart=corr_df.head(5)
        corr_df.sort_values(by=['Correlation Coefficient (r)'], inplace=True)
        bottom_chart=corr_df.head(5)
    else:
        #If Outpatient remove 'Residential or Inpatient' questions before sorting
        rp = "Residential or Inpatient"
        ip_q = q_index[q_index["area"] == rp]['content']
        exclude = pd.DataFrame({'Items':[x for x in ip_q]})
        corr_op_df = corr_df[~corr_df.Items.isin(exclude.Items)]
        corr_op_df.sort_values(by=['Correlation Coefficient (r)'], inplace=True, ascending=False)
        top_chart=corr_op_df.head(5)
        corr_op_df.sort_values(by=['Correlation Coefficient (r)'], inplace=True)
        bottom_chart=corr_op_df.head(5)    
    return top_chart, bottom_chart

In [ ]:
#Function to add slides

#Adds title slide, contents slide and OPOC Respondent numbers for report group (3 slides total)
def add_summary_slides(group_data, summary_data, prs, report_month, report_year, report_group):
    #Change to correct label to be displayed in Respondent numbers table
    fam_label = 'Registered Client who is a family member/significant other/supporter of a person with a mental health, substance abuse, addiction, and/or gambling-related problems'
    mh_label = 'Registered Clients with mental health, substance abuse, addiction, and/or gambling-related problems'
    #Generate table for Repsondent Numbers
    if report_group == 'Non-Registered Family':
        tmp = pd.DataFrame(summary_data.encounter_type.value_counts())
        ov_sum = pd.Series(sum(tmp['encounter_type']),index=['encounter_type'])
        ov_sum = ov_sum.rename('Overall (All programs)')
        tmp = tmp.append(ov_sum)
        tmp.index.names=[' ']
        counts_tab = tmp.rename(columns={'encounter_type':'Non-Registered client who is a family member/significant other/supporter of a person with a mental health, substance abuse, addiction, and/or gambling-related problems'}, index={"IP":"Inpatient", "OP":"Outpatient", "Total":"Overall (All programs)"})
    else:
        counts_tab = pd.crosstab(index=summary_data['encounter_type'], columns=summary_data['Client Type'],colnames=[''],rownames=[' '],margins=True, margins_name='Total').rename(columns={"FAM":fam_label,"MH":mh_label}, index={"IP":"Inpatient", "OP":"Outpatient", "Total":"Overall (All programs)"})    
    try:
        if (report_group == 'Non-Registered Family') or (report_group == 'Registered Family') or counts_tab['Total']['Inpatient'] < 50:
            group = ["OP"]
        else:
            group = ["IP", "OP"]   
    except:
        group = ["OP"] 
    counts_tab = counts_tab.rename(columns={'Total':"Total number of respondents"})
    #Add title slide
    title_slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(title_slide_layout)
    title = slide.shapes.title
    title.text = "{0} Ontario Perception of CARE (OPOC): {1}\n\nTop Bottom Item Analysis &\nCorrelational Analysis".format(report_year,report_group) 
    placeholder = slide.placeholders[10]
    report_date = str(int(report_year)+1)
    placeholder.text = report_month + ' ' + report_date
    #Add Contents slide
    if report_group == 'Non-Registered Family':
        layout = prs.slide_layouts[12]
        slide = prs.slides.add_slide(layout)
    else:
        layout = prs.slide_layouts[1]
        slide = prs.slides.add_slide(layout)
    #Add OPOC Respondents Numbers
    layout = prs.slide_layouts[5]
    slide = prs.slides.add_slide(layout)
    title = slide.shapes.title
    title.text = "OPOC Respondent Numbers ({0})".format(report_group)
    if report_group == 'Non-Registered Family':
        top, left, width, height = Inches(1.92), Inches(2.08), Inches(5.75), Inches(4)
    else:
        top, left, width, height = Inches(1.94), Inches(0.58), Inches(8.83), Inches(4)
    #Add Table
    shp = df_to_table(slide, counts_tab.reset_index(), left, top, width, height)
    table = shp.table
    #Adjust text formatting
    for x in range(0,len(table.rows)):
        for y in range(0,len(table.columns)):
            table.cell(x,y).vertical_anchor = MSO_ANCHOR.MIDDLE
            paragraph = table.cell(x,y).text_frame.paragraphs[0]
            paragraph.font.size = Pt(12)
            if y > 0:
                paragraph.alignment = PP_ALIGN.CENTER
            else: 
                continue
    #Adjust height of rows
    table.rows[0].height = Inches(1.95)
    for x in range(1,len(table.rows)):
        table.rows[x].height = Inches(0.62)
    if report_group == 'Non-Registered Family':
        table.columns[0].width = Inches(1.64)
        table.columns[1].width = Inches(4.2)
    else:
        table.columns[0].width = Inches(1.17)
        table.columns[1].width = Inches(3)
    try:
        table.columns[3].width = Inches(1.6)
        table.columns[2].width = Inches(3)
    except IndexError:
        return prs, group
    return prs, group

In [ ]:
#Adds demographic slides - "Gender", "Age Group", "Population Group", "Sexual Orientation", "Mother Tongue" (5 slides total)
def add_demographic_slides(report_group, group_data, prs, group):
    #Identify demographic chart topics
    if (report_group == 'Registered Family') or (report_group == 'Non-Registered Family'):
        demographic_topics = ["Family Mbr Type", "Gender", "Age Group", "Population Group", "Sexual Orientation", "Mother Tongue"]
    else:
        demographic_topics = ["Gender", "Age Group", "Population Group", "Sexual Orientation", "Mother Tongue"]
    for topic in demographic_topics:
        #Generate table data
        updated_chart = demographic_table(group_data, topic)
        #Add slide
        layout = prs.slide_layouts[5]
        slide = prs.slides.add_slide(layout)
        title = slide.shapes.title
        title.text = "Demographics"
        #Generate chart bars
        chart_data = ChartData()
        chart_data.categories = updated_chart.index
        if report_group == 'Registered Family':
            chart_data.add_series('Outpatient', updated_chart['Outpatient'].values)
        else:
            try:
                chart_data.add_series('Inpatient', updated_chart['Inpatient'].values)
                chart_data.add_series('Outpatient', updated_chart['Outpatient'].values)
            except:
                chart_data.add_series('Outpatient', updated_chart['Outpatient'].values)
        #Chart position
        x, y, cx, cy = Inches(1), Inches(1.83), Inches(8), Inches(4.42)    
        #Add chart
        try:
            graphic_frame = slide.shapes.add_chart(XL_CHART_TYPE.COLUMN_CLUSTERED, x, y, cx, cy, chart_data)
        except:
            chart_data = ChartData()
            chart_data.categories = updated_chart.index
            chart_data.add_series('Outpatient', updated_chart['Outpatient'].values)
            graphic_frame = slide.shapes.add_chart(XL_CHART_TYPE.COLUMN_CLUSTERED, x, y, cx, cy, chart_data)
        chart = graphic_frame.chart
        if len(group) == 1:
            chart.plots[0].vary_by_categories = False
        #Remove gridlines
        value_axis = chart.value_axis
        value_axis.has_major_gridlines = False
        #chart formatting
        category_axis = chart.category_axis
        category_axis.tick_labels.font.size = Pt(10)
        category_axis.has_major_gridlines = False
        value_axis = chart.value_axis
        value_axis.maximum_scale = 1
        value_axis.tick_labels.font.size = Pt(10)
        #chart labels
        chart.plots[0].has_data_labels = True
        data_labels = chart.plots[0].data_labels
        data_labels.number_format = '0%'
        data_labels.font.size = Pt(8)
        data_labels.position = XL_LABEL_POSITION.OUTSIDE_END
        #chart legend
        chart.has_legend = True
        chart.legend.position = XL_LEGEND_POSITION.RIGHT
        chart.legend.include_in_layout = False
        chart.legend.font.size = Pt(10)
        #add chart titles
        chart.has_title
        chart.chart_title.has_text_frame = True
        chart.chart_title.text_frame.text = topic
        p = chart.chart_title.text_frame.paragraphs[0]
        p.font.bold = True
        axis =  chart.value_axis
        axis.has_title
        axis.axis_title.has_text_frame = True
        axis.axis_title.text_frame.text = 'Percent'
        p = axis.axis_title.text_frame.paragraphs[0]
        p.font.size = Pt(10)
    return prs

In [ ]:
#Adds Top 10 and Bottom 10 slides for Inpatient and Outpatient (9 slides total):
#Methodology x1 
#Top Items, Bottom Items, Overview, Domains x2 (IP and OP) 
def add_tb_slides(group_data, prs, report_year, report_group, q_index, group):
    #add methodology slide
    if report_group == 'Non-Registered Family':
        tb_num = 5
        layout = prs.slide_layouts[13]
    else:
        tb_num = 10
        layout = prs.slide_layouts[2]
    slide = prs.slides.add_slide(layout)
    #track top and bottom 10 items for conclusion
    labels = ["item", "rank_tb", 'type']
    conclusion = pd.DataFrame(columns = labels)
    value=[]
    type_value = []
    ranking = []
    for i in group:
        if i == "IP":
            group_name = " Inpatient"
        else:
            group_name = " Outpatient"
        if report_group == 'Non-Registered Family':
            group_name = ' (All Programs)'
        #Generate tables for top chart and bottom chart
        top_chart, bottom_chart, n, miss = tb_table(group_data, i, q_index, report_group)
        #gather data for conclusion
        for k in top_chart['Items']:
            value.append(q_index[q_index['content']==k]['question_no'].values[0])
            type_value.append(i)
            ranking.append('top')
        for k in bottom_chart['Items']:
            value.append(q_index[q_index['content']==k]['question_no'].values[0])
            type_value.append(i)
            ranking.append('bottom')
        
        #Top 10 Items slides
        layout = prs.slide_layouts[6]
        slide = prs.slides.add_slide(layout)
        title = slide.shapes.title
        title.text = "{0} Top {3} {1}{2} Items - highest to lowest".format(report_year, report_group, group_name, tb_num)
        top, left, width, height = Inches(1.25), Inches(0.25), Inches(9.5), Inches(4)
        shp = df_to_table(slide, top_chart, left, top, width, height)
        table = shp.table
        #Format Header
        cell = table.cell(0,1)
        other_cell = table.cell(0,2)
        cell.merge(other_cell)
        cell.text = "Positive Responses (Of those applicable)"
        cell = table.cell(0,4)
        other_cell = table.cell(0,5)
        cell.merge(other_cell)
        cell.text = "N/A & Missing (Of overall)"
        #Keep track of top items in each domain for domain slide
        top_domain = []
        #add domain to each question label
        for x in range(1,len(table.rows)):
            paragraph = table.cell(x,0).text_frame.paragraphs[0]
            run = paragraph.add_run()
            tmp = q_index[q_index["content"]==paragraph.text]["area"].values[0]
            run.text = " ({0}) ".format(tmp)
            top_domain.append(tmp)
            font = run.font
            font.bold = True
        #Adjust text formatting
        for y in range(0,len(table.columns)):
            table.cell(0,y).vertical_anchor = MSO_ANCHOR.MIDDLE
            paragraph = table.cell(0,y).text_frame.paragraphs[0]
            paragraph.font.size = Pt(12)
            if y > 0:
                    paragraph.alignment = PP_ALIGN.CENTER
            else: 
                continue
        for x in range(1,len(table.rows)):
            for y in range(0,len(table.columns)):
                table.cell(x,y).vertical_anchor = MSO_ANCHOR.MIDDLE
                paragraph = table.cell(x,y).text_frame.paragraphs[0]
                paragraph.font.size = Pt(10.5)
                if y > 0:
                    paragraph.alignment = PP_ALIGN.CENTER
                else: 
                    continue
        #Adjust height of rows
        table.rows[0].height = Inches(0.50)
        for x in range(1,len(table.rows)):
            table.rows[x].height = Inches(0.1)
        table.columns[0].width = Inches(5)
        table.columns[1].width = Inches(0.8)
        table.columns[2].width = Inches(0.8)
        table.columns[3].width = Inches(1.3)
        table.columns[4].width = Inches(0.8)
        table.columns[5].width = Inches(0.8)

        #Bottom 10 Items
        layout = prs.slide_layouts[6]
        slide = prs.slides.add_slide(layout)
        title = slide.shapes.title
        title.text = "{0} Bottom {3} {1}{2} Items - lowest to highest".format(report_year, report_group, group_name,tb_num)
        top, left, width, height = Inches(1.25), Inches(0.25), Inches(9.5), Inches(4)
        shp = df_to_table(slide, bottom_chart, left, top, width, height)
        table = shp.table
        #Format Header
        cell = table.cell(0,1)
        other_cell = table.cell(0,2)
        cell.merge(other_cell)
        cell.text = "Positive Responses (Of those applicable)"
        cell = table.cell(0,4)
        other_cell = table.cell(0,5)
        cell.merge(other_cell)
        cell.text = "N/A & Missing (Of overall)"
        #Keep track of bottom items in each domain for domain slide
        bottom_domain = []
        #add domain to each question label
        for x in range(1,len(table.rows)):
            paragraph = table.cell(x,0).text_frame.paragraphs[0]
            run = paragraph.add_run()
            tmp = q_index[q_index["content"]==paragraph.text]["area"].values[0]
            run.text = " ({0}) ".format(tmp)
            bottom_domain.append(tmp)
            font = run.font
            font.bold = True
        #Adjust text formatting
        for y in range(0,len(table.columns)):
            table.cell(0,y).vertical_anchor = MSO_ANCHOR.MIDDLE
            paragraph = table.cell(0,y).text_frame.paragraphs[0]
            paragraph.font.size = Pt(12)
            if y > 0:
                    paragraph.alignment = PP_ALIGN.CENTER
            else: 
                continue
        for x in range(1,len(table.rows)):
            for y in range(0,len(table.columns)):
                table.cell(x,y).vertical_anchor = MSO_ANCHOR.MIDDLE
                paragraph = table.cell(x,y).text_frame.paragraphs[0]
                paragraph.font.size = Pt(10.5)
                if y > 0:
                    paragraph.alignment = PP_ALIGN.CENTER
                else: 
                    continue
        #Adjust height of rows
        table.rows[0].height = Inches(0.50)
        for x in range(1,len(table.rows)):
            table.rows[x].height = Inches(0.1)
        table.columns[0].width = Inches(5)
        table.columns[1].width = Inches(0.8)
        table.columns[2].width = Inches(0.8)
        table.columns[3].width = Inches(1.3)
        table.columns[4].width = Inches(0.8)
        table.columns[5].width = Inches(0.8)

        #Add Top/Bottom Overview slide
        layout = prs.slide_layouts[7]
        slide = prs.slides.add_slide(layout)
        title = slide.shapes.title
        title.text = "Top/Bottom {2} {0}{1} Overview".format(report_group, group_name, tb_num)
        placeholder = slide.placeholders[10]
        paragraph = placeholder.text_frame
        run = paragraph.paragraphs[0].add_run()
        num = top_chart['Positive Responses (Of those applicable)']
        top_low = num.values[len(num)-1]
        top_high = num.values[0]
        if report_group == 'Non-Registered Family':
            run.text = "The top {2} positive responses ranged from {0} to {1}.".format(top_low, top_high, tb_num)
        else:
            run.text = "The top {3} positive responses for{0} ranged from {1} to {2}.".format(group_name, top_low, top_high, tb_num)
        run = paragraph.add_paragraph()
        run.space_after = Pt(14)
        run.space_before = Pt(14)
        num = bottom_chart['Positive Responses (Of those applicable)']
        bot_low = num.values[0]
        bot_high = num.values[len(num)-1]
        if report_group == 'Non-Registered Family':
            run.text = "The bottom {2} positive responses ranged from {0} to {1}.".format(bot_low, bot_high, tb_num)
        else:
            run.text = "The bottom {3} positive responses for{0} ranged from {1} to {2}.".format(group_name, bot_low, bot_high, tb_num)
        top_item = top_chart['Items'].values[0]
        area = q_index[q_index["content"]==top_item]["area"].values[0]
        run = paragraph.add_paragraph()
        run.space_after = Pt(14)
        run.text = "The top rated positive response was item {0} ({2}) – with {1}.".format(top_item, top_high, area)
        bot_item = bottom_chart['Items'].values[0]
        area = q_index[q_index["content"]==bot_item]["area"].values[0]
        run = paragraph.add_paragraph()
        run.space_after = Pt(14)
        run.text = "The lowest rated positive response was item {0} ({2}) – with {1}.".format(bot_item, bot_low, area)
        run = paragraph.add_paragraph()
        run.space_after = Pt(14)
        miss_str = ""
        for x in miss:
            miss_str += x + " "
        run.text = "The N/A & Missing column were ≥ 10% of the overall {1}{2} respondents for items: {0}".format(miss_str, n, group_name)
        paragraph = placeholder.text_frame.paragraphs
        for x in paragraph:
            x.font.size = Pt(20)
            
        #Add Domain Slide
        layout = prs.slide_layouts[8]
        slide = prs.slides.add_slide(layout)
        title = slide.shapes.title
        title.text = "{0} Highest and Lowest Rated Item Domains ({1}{2})".format(report_year, report_group, group_name)
        #Top Domains percentages
        top_domain_count = dict([[x,top_domain.count(x)/tb_num] for x in top_domain])
        top_domain_perc = pd.DataFrame.from_dict(top_domain_count, orient = 'index')
        top_domain_perc.sort_values(by=[0], inplace=True, ascending=False)
        #Add pie chart
        placeholder = slide.placeholders[15]
        chart_data = ChartData()
        chart_data.categories = top_domain_perc.index
        chart_data.add_series('Percent', top_domain_perc[0])
        chart = placeholder.insert_chart(XL_CHART_TYPE.PIE, chart_data).chart
        #Add legend
        chart.has_legend = True
        chart.legend.position = XL_LEGEND_POSITION.RIGHT
        chart.legend.include_in_layout = False
        chart.legend.font.size = Pt(10)
        #Add title
        chart.has_title
        chart.chart_title.has_text_frame = True
        chart.chart_title.text_frame.text = "Highest Rated Domains"
        chart.chart_title.text_frame.paragraphs[0].font.size = Pt(16)
        p = chart.chart_title.text_frame.paragraphs[0]
        p.font.bold = True
        #add labels
        chart.plots[0].has_data_labels = True
        data_labels = chart.plots[0].data_labels
        data_labels.font.size = Pt(10)
        data_labels.number_format = '0%'
        data_labels.position = XL_LABEL_POSITION.OUTSIDE_END
        
        #Add top domain text
        placeholder = slide.placeholders[19]
        paragraph = placeholder.text_frame
        run = paragraph.paragraphs[0].add_run()
        run.text = "When looking at the {0} highest rated items:".format(tb_num)
        paragraph = placeholder.text_frame.paragraphs
        paragraph[0].font.size = Pt(12)
        placeholder = slide.placeholders[14]
        paragraph = placeholder.text_frame
        for area in top_domain_perc.index:
            run = paragraph.add_paragraph()
            run.text = "{0}% were from the {1} domain".format(int(top_domain_perc.loc[area,0]*100), area)
        paragraph = placeholder.text_frame.paragraphs
        for x in paragraph:
            x.font.size = Pt(12)
        
        #Bottom Domains percentages
        bottom_domain_count = dict([[x,bottom_domain.count(x)/tb_num] for x in bottom_domain])
        bottom_domain_perc = pd.DataFrame.from_dict(bottom_domain_count, orient = 'index')
        bottom_domain_perc.sort_values(by=[0], inplace=True, ascending=False)
        #Add pie chart
        placeholder = slide.placeholders[16]
        chart_data = ChartData()
        chart_data.categories = bottom_domain_perc.index
        chart_data.add_series('Percent', bottom_domain_perc[0])
        chart = placeholder.insert_chart(XL_CHART_TYPE.PIE, chart_data).chart
        #add legend
        chart.has_legend = True
        chart.legend.position = XL_LEGEND_POSITION.RIGHT
        chart.legend.include_in_layout = False
        chart.legend.font.size = Pt(10)
        #add title
        chart.has_title
        chart.chart_title.has_text_frame = True
        chart.chart_title.text_frame.text = "Lowest Rated Domains"
        chart.chart_title.text_frame.paragraphs[0].font.size = Pt(16)
        p = chart.chart_title.text_frame.paragraphs[0]
        p.font.bold = True
        chart.plots[0].has_data_labels = True
        #add labels
        data_labels = chart.plots[0].data_labels
        data_labels.font.size = Pt(10)
        data_labels.number_format = '0%'
        data_labels.position = XL_LABEL_POSITION.OUTSIDE_END
        #add bottom domains text
        placeholder = slide.placeholders[20]
        paragraph = placeholder.text_frame
        run = paragraph.paragraphs[0].add_run()
        run.text = "When looking at the bottom {0} positive responses:". format(tb_num)
        paragraph = placeholder.text_frame.paragraphs
        paragraph[0].font.size = Pt(12)
        placeholder = slide.placeholders[18]
        paragraph = placeholder.text_frame
        for area in bottom_domain_perc.index:
            run = paragraph.add_paragraph()
            run.text = "{0}% were from the {1} domain".format(int(bottom_domain_perc.loc[area,0]*100), area)
        paragraph = placeholder.text_frame.paragraphs
        for x in paragraph:
            x.font.size = Pt(12)
    #populate conclusion table for conclusion slide
    conclusion['item'] = value
    conclusion['type'] = type_value
    conclusion['rank_tb'] = ranking
    return prs, conclusion

In [ ]:
#Adds Overall slides for Strongly Agree and Agree and Strongly Agree only (2 slides total):
def add_overall_slides(group_data, prs, report_year, report_group, q_index, q_overall, group):
    #Strongly Agree and Agree Slide
    layout = prs.slide_layouts[9]
    slide = prs.slides.add_slide(layout)
    title = slide.shapes.title
    title.text = "{0} Overall Experience Chart - Agree & Strongly Agree".format(report_group)
    #Generate tables 
    chart_34, chart_4 = overall_tables(group_data, report_group, q_index, q_overall, group)
    #Add Strongly Agree and Agree chart 
    placeholder = slide.placeholders[15]
    chart_data = ChartData()
    chart_data.categories = chart_34.columns
    if len(group) == 1:
        chart_data.add_series('Outpatient Positive Responses (%)', chart_34.loc['Outpatient'])
    else:
        chart_data.add_series('Inpatient Positive Responses (%)', chart_34.loc['Inpatient'])
        chart_data.add_series('Outpatient Positive Responses (%)', chart_34.loc['Outpatient'])
    chart = placeholder.insert_chart(XL_CHART_TYPE.BAR_CLUSTERED, chart_data).chart
    chart.plots[0].vary_by_categories = False
    #Add legend
    chart.has_legend = True
    chart.legend.position = XL_LEGEND_POSITION.BOTTOM
    chart.legend.include_in_layout = False
    chart.legend.font.size = Pt(10)
    #add titles
    chart.has_title
    chart.chart_title.has_text_frame = True
    chart.chart_title.text_frame.text = "{0} Overall Experience - OPOC {1}".format(report_group, report_year)
    chart.chart_title.text_frame.paragraphs[0].font.size = Pt(16)
    p = chart.chart_title.text_frame.paragraphs[0]
    p.font.bold = True
    #Add data labels
    chart.plots[0].has_data_labels = True
    data_labels = chart.plots[0].data_labels
    data_labels.font.size = Pt(10)
    data_labels.number_format = "0.0%"
    data_labels.position = XL_LABEL_POSITION.OUTSIDE_END
    #chart formatting
    category_axis = chart.category_axis
    category_axis.tick_labels.font.size = Pt(10)
    category_axis.has_major_gridlines = False
    category_axis.format.line.color.rgb = RGBColor(217, 217, 217)
    value_axis = chart.value_axis
    value_axis.maximum_scale = 1.2
    value_axis.tick_labels.font.size = Pt(10)
    value_axis.has_major_gridlines = True
    value_axis.format.line.color.rgb = RGBColor(255, 255, 255)
    value_axis.major_gridlines.format.line.color.rgb = RGBColor(217, 217, 217)
    #Recap Paragraph at the bottom of the slide
    placeholder = slide.placeholders[11]
    paragraph = placeholder.text_frame
    run = paragraph.paragraphs[0].add_run()
    high_group = max(chart_34.index)
    low_group = min(chart_34.index)
    high_group_high_q = q_overall[q_overall['content']==chart_34.idxmax(axis=1)[high_group]]['question_no'].values[0]
    high_group_high = round(max(chart_34[chart_34.index == high_group].values[0])*100,2)
    high_group_low_q = q_overall[q_overall['content']==chart_34.idxmin(axis=1)[high_group]]['question_no'].values[0]
    high_group_low = round(min(chart_34[chart_34.index == high_group].values[0])*100,2)
    low_group_high_q = q_overall[q_overall['content']==chart_34.idxmax(axis=1)[low_group]]['question_no'].values[0]
    low_group_high = round(max(chart_34[chart_34.index == low_group].values[0])*100,2)
    low_group_low_q = q_overall[q_overall['content']==chart_34.idxmin(axis=1)[low_group]]['question_no'].values[0]
    low_group_low = round(min(chart_34[chart_34.index == low_group].values[0])*100,2)
    if len(group) == 1:
        run.text = "The highest positive response for {0} was {1} ({2}%) while the lowest positive response for {0} was {3} ({4}%).".format(high_group, high_group_high_q, high_group_high, high_group_low_q, high_group_low)
    else:
        run.text = "Overall, {0} positive responses were higher than {1} positive responses for all questions asked in OPOC’s overall experience domain. The highest positive response for {0} was {2} ({3}%) and for {1} was {4} ({5}%).  The lowest positive response for {0} was {6} ({7}%) and for {1} was {8} ({9}%).".format(high_group, low_group, high_group_high_q, high_group_high, low_group_high_q, low_group_high, high_group_low_q, high_group_low, low_group_low_q, low_group_low)
    paragraph = placeholder.text_frame.paragraphs
    paragraph[0].font.size = Pt(14)
    
    #Strongly Agree Slide
    layout = prs.slide_layouts[9]
    slide = prs.slides.add_slide(layout)
    title = slide.shapes.title
    title.text = "{0} Overall Experience Chart - Strongly Agree".format(report_group)
    #Add chart
    placeholder = slide.placeholders[15]
    chart_data = ChartData()
    chart_data.categories = chart_4.columns
    if len(group) == 1:
        chart_data.add_series('Outpatient Strongly Agree Positive Responses (%)', chart_4.loc['Outpatient'])
    else:
        chart_data.add_series('Inpatient Strongly Agree Positive Responses (%)', chart_4.loc['Inpatient'])
        chart_data.add_series('Outpatient Strongly Agree Positive Responses (%)', chart_4.loc['Outpatient'])
    chart = placeholder.insert_chart(XL_CHART_TYPE.BAR_CLUSTERED, chart_data).chart
    chart.plots[0].vary_by_categories = False
    #add legend
    chart.has_legend = True
    chart.legend.position = XL_LEGEND_POSITION.BOTTOM
    chart.legend.include_in_layout = False
    chart.legend.font.size = Pt(10)
    #add title
    chart.has_title
    chart.chart_title.has_text_frame = True
    chart.chart_title.text_frame.text = "{0} Overall Experience - OPOC {1} Strongly Agree Responses".format(report_group, report_year)
    chart.chart_title.text_frame.paragraphs[0].font.size = Pt(16)
    p = chart.chart_title.text_frame.paragraphs[0]
    p.font.bold = True
    chart.plots[0].has_data_labels = True
    #add data labels
    data_labels = chart.plots[0].data_labels
    data_labels.font.size = Pt(10)
    data_labels.number_format = "0.0%"
    data_labels.position = XL_LABEL_POSITION.OUTSIDE_END
    #chart formatting
    category_axis = chart.category_axis
    category_axis.tick_labels.font.size = Pt(10)
    category_axis.has_major_gridlines = False
    category_axis.format.line.color.rgb = RGBColor(217, 217, 217)
    value_axis = chart.value_axis
    value_axis.maximum_scale = 1.2
    value_axis.tick_labels.font.size = Pt(10)
    value_axis.has_major_gridlines = True
    value_axis.format.line.color.rgb = RGBColor(255, 255, 255)
    value_axis.major_gridlines.format.line.color.rgb = RGBColor(217, 217, 217)
    #Recap Paragraph at the bottom of the slide
    placeholder = slide.placeholders[11]
    paragraph = placeholder.text_frame
    run = paragraph.paragraphs[0].add_run()
    high_group = max(chart_4.index)
    low_group = min(chart_4.index)
    high_group_high_q = q_overall[q_overall['content']==chart_4.idxmax(axis=1)[high_group]]['question_no'].values[0]
    high_group_high = round(max(chart_4[chart_4.index == high_group].values[0])*100,2)
    high_group_low_q = q_overall[q_overall['content']==chart_4.idxmin(axis=1)[high_group]]['question_no'].values[0]
    high_group_low = round(min(chart_4[chart_4.index == high_group].values[0])*100,2)
    low_group_high_q = q_overall[q_overall['content']==chart_4.idxmax(axis=1)[low_group]]['question_no'].values[0]
    low_group_high = round(max(chart_4[chart_4.index == low_group].values[0])*100,2)
    low_group_low_q = q_overall[q_overall['content']==chart_4.idxmin(axis=1)[low_group]]['question_no'].values[0]
    low_group_low = round(min(chart_4[chart_4.index == low_group].values[0])*100,2)
    if len(group) == 1:
        run.text = "The highest strongly agree response for {0} was {1} ({2}%) while the lowest positive response for {0} was {3} ({4}%).".format(high_group, high_group_high_q, high_group_high, high_group_low_q, high_group_low)
    else:
        run.text = "Overall, {0} strongly agree responses were higher than {1} responses for all questions asked in OPOC’s overall experience domain. The highest strongly agree response for {0} was {2} ({3}%) and for {1} was {4} ({5}%).  The lowest strongly agree response for {0} was {6} ({7}%) and for {1} was {8} ({9}%).".format(high_group, low_group, high_group_high_q, high_group_high, low_group_high_q, low_group_high, high_group_low_q, high_group_low, low_group_low_q, low_group_low)
    paragraph = placeholder.text_frame.paragraphs
    paragraph[0].font.size = Pt(14)
    return prs

In [ ]:
#Adds Non-Registered Family Overall slides for Strongly Agree and Agree and Strongly Agree only (2 slides total):
def add_nonreg_overall_slides(group_data, prs, report_year, report_group, q_index, q_overall, group):
    #Strongly Agree and Agree Slide
    layout = prs.slide_layouts[9]
    slide = prs.slides.add_slide(layout)
    title = slide.shapes.title
    title.text = "Overall Experience Chart - Agree & Strongly Agree"
    #Generate tables 
    chart_34, chart_4 = overall_tables(group_data, report_group, q_index, q_overall, group)
    #load previous data
    wb_nonreg = pd.ExcelFile('nonreg_overall_data.xlsx')
    data = wb_nonreg.parse('Sheet1')
    data = data.set_index('Items', drop=True)
    pos_resp = chart_34.iloc[0]['17. I think the services provided here are of high quality.']
    diff = round((pos_resp - data[(int(report_year) - 1)].values[0])*100,1)
    data[int(report_year)] = pos_resp
    sa = '{0} Strongly Agree Reponses (%)'.format(report_year)
    pos = '{0} Positive Reponses (%)'.format(report_year)
    tmp = [0]
    strongly_agree = pd.DataFrame(0, index=tmp, columns=[sa,pos])
    strongly_agree[sa] = chart_4.iloc[0]['17. I think the services provided here are of high quality.']
    strongly_agree[pos] = pos_resp
    data.to_excel('nonreg_overall_data.xlsx')
    #Add Strongly Agree and Agree chart 
    placeholder = slide.placeholders[15]
    chart_data = ChartData()
    chart_data.categories = data.index
    for i in data.columns:
        chart_data.add_series('{0} Positive Reponses (%)'.format(i), data[i])
    chart = placeholder.insert_chart(XL_CHART_TYPE.BAR_CLUSTERED, chart_data).chart
    chart.plots[0].vary_by_categories = False
    #Add legend
    chart.has_legend = True
    chart.legend.position = XL_LEGEND_POSITION.BOTTOM
    chart.legend.include_in_layout = False
    chart.legend.font.size = Pt(10)
    #add titles
    chart.has_title
    chart.chart_title.has_text_frame = True
    chart.chart_title.text_frame.text = "Overall Experience - Positive Response"
    chart.chart_title.text_frame.paragraphs[0].font.size = Pt(16)
    p = chart.chart_title.text_frame.paragraphs[0]
    p.font.bold = True
    #Add data labels
    chart.plots[0].has_data_labels = True
    data_labels = chart.plots[0].data_labels
    data_labels.font.size = Pt(10)
    data_labels.number_format = "0.0%"
    data_labels.position = XL_LABEL_POSITION.OUTSIDE_END
    #chart formatting
    category_axis = chart.category_axis
    category_axis.tick_labels.font.size = Pt(10)
    category_axis.has_major_gridlines = False
    category_axis.format.line.color.rgb = RGBColor(217, 217, 217)
    value_axis = chart.value_axis
    value_axis.maximum_scale = 1
    value_axis.minimum_scale = 0.8
    value_axis.tick_labels.font.size = Pt(10)
    value_axis.has_major_gridlines = True
    value_axis.format.line.color.rgb = RGBColor(255, 255, 255)
    value_axis.major_gridlines.format.line.color.rgb = RGBColor(217, 217, 217)
    #Recap Paragraph at the bottom of the slide
    placeholder = slide.placeholders[11]
    paragraph = placeholder.text_frame
    run = paragraph.paragraphs[0].add_run()
    high_group = max(chart_34.index)
    if diff > 0:
        change = 'increase'
    else:
        change = 'decrease'
    run.text = "For the Non-Registered Family survey, there was only 1 overall experience question (item 17 which is equivalent to item 31 in the Registered OPOC) and the positive response rate was {0}% (a {1}% {2} compared to {3} results).".format(round(pos_resp*100,1), abs(diff), change, (int(report_year)-1))
    paragraph = placeholder.text_frame.paragraphs
    paragraph[0].font.size = Pt(14)
    
    #Strongly Agree Slide
    layout = prs.slide_layouts[9]
    slide = prs.slides.add_slide(layout)
    title = slide.shapes.title
    title.text = "Overall Experience Chart - Strongly Agree"
    #Add chart
    placeholder = slide.placeholders[15]
    chart_data = ChartData()
    chart_data.categories = strongly_agree.columns
    chart_data.add_series('Series 1', strongly_agree.iloc[0].values)
    chart = placeholder.insert_chart(XL_CHART_TYPE.COLUMN_CLUSTERED, chart_data).chart
    chart.plots[0].vary_by_categories = False
    #add legend
    chart.has_legend = False
    #add title
    chart.has_title
    chart.chart_title.has_text_frame = True
    chart.chart_title.text_frame.text = "Overall Experience - Strongly Agree Responses"
    chart.chart_title.text_frame.paragraphs[0].font.size = Pt(16)
    p = chart.chart_title.text_frame.paragraphs[0]
    p.font.bold = True
    chart.plots[0].has_data_labels = True
    #add data labels
    data_labels = chart.plots[0].data_labels
    data_labels.font.size = Pt(10)
    data_labels.number_format = "0.0%"
    data_labels.position = XL_LABEL_POSITION.OUTSIDE_END
    #chart formatting
    category_axis = chart.category_axis
    category_axis.tick_labels.font.size = Pt(10)
    category_axis.has_major_gridlines = False
    category_axis.format.line.color.rgb = RGBColor(217, 217, 217)
    value_axis = chart.value_axis
    value_axis.maximum_scale = 1
    value_axis.tick_labels.font.size = Pt(10)
    value_axis.has_major_gridlines = True
    value_axis.format.line.color.rgb = RGBColor(255, 255, 255)
    value_axis.major_gridlines.format.line.color.rgb = RGBColor(217, 217, 217)
    #Recap Paragraph at the bottom of the slide
    placeholder = slide.placeholders[11]
    paragraph = placeholder.text_frame
    run = paragraph.paragraphs[0].add_run()
    run.text = "The strongly agree response for question 17 is {0}%.".format(round((strongly_agree[sa].values[0])*100,1))
    paragraph = placeholder.text_frame.paragraphs
    paragraph[0].font.size = Pt(14)
    paragraph[0].alignment = PP_ALIGN.CENTER
    return prs

In [ ]:
#Adds Top 5 and Bottom 5 Correlated Items for each Overall Question for Inpatient and Outpatient (15 slides total):
#Methodology x1 
#Q30, Q31, Q32, Overview, Conclusion Summary, Conclusion, final conclusion x2 (IP and OP) 
def add_corr_slides(group_data, prs, report_year, report_group, q_index, q_overall, conclusion, group):
    #Add methodology slide
    if report_group == 'Non-Registered Family':
        layout = prs.slide_layouts[15]
    else:
        layout = prs.slide_layouts[3]
    slide = prs.slides.add_slide(layout)
    overall_q = q_overall['question_no']
    for i in group:  
        if i == "IP":
            group_name = " Inpatient"
        else:
            group_name = " Outpatient"
        if report_group == 'Non-Registered Family':
            group_name = ' (All Programs)'
        #Track top and bottom 5 items for conclusion
        labels = ["item", "rank_corr", 'type', 'q_no']
        conclusion_corr = pd.DataFrame(columns = labels)
        q_value=[]
        type_value = []
        ranking = []
        q_no = []
        overview = pd.DataFrame(columns=overall_q)
        for q in overall_q:
            #Add correlation slide
            layout = prs.slide_layouts[10]
            slide = prs.slides.add_slide(layout)
            title = slide.shapes.title
            if report_group == 'Non-Registered Family':
                title.text = "{0} Top/Bottom Correlated Items: Overall Experience ({2}{3})".format(report_year, q, report_group, group_name)
            else:    
                title.text = "{0} Top/Bottom Correlated Items: Overall Experience {1} ({2}{3})".format(report_year, q, report_group, group_name)
            placeholder = slide.placeholders[11]
            paragraph = placeholder.text_frame
            run = paragraph.paragraphs[0].add_run()
            run.text = q_overall.loc[q_overall['question_no']==q,'content'].iloc[0] 
            #Generate table data
            top_chart, bottom_chart = corr_tables(group_data, i, q_index, q)
            #Track top and bottom 5 for conclusion
            for k in top_chart['Items']:
                q_value.append(q_index[q_index['content']==k]['question_no'].values[0])
                type_value.append(i)
                ranking.append('top')
                q_no.append(q)
            for k in bottom_chart['Items']:
                q_value.append(q_index[q_index['content']==k]['question_no'].values[0])
                type_value.append(i)
                ranking.append('bottom')
                q_no.append(q)
            #Track data for Overview slide
            if top_chart.empty == True:
                run.text = 'Not applicable due to small sample size'
                top_value = 'N/A'
                bottom_value = 'N/A'
                top_item = 'N/A'
                bottom_item = 'N/A'
            else:
                top_value = top_chart['Correlation Coefficient (r)'].values[0]
                bottom_value = bottom_chart['Correlation Coefficient (r)'].values[0]
                top_item = q_index[q_index['content']==top_chart['Items'].values[0]]['question_no'].values[0]
                bottom_item = q_index[q_index['content']==bottom_chart['Items'].values[0]]['question_no'].values[0]
            #Add Top table
            top, left, width, height = Inches(1.5), Inches(0.75), Inches(8.5), Inches(2.33)
            shp = df_to_table(slide, top_chart, left, top, width, height)
            table = shp.table
            cell = table.cell(0,0)
            cell.text = "Top 5 Correlated Items"
            #track items in each domain for overview slide
            top_domain = []
            for x in range(1,len(table.rows)):
                paragraph = table.cell(x,0).text_frame.paragraphs[0]
                run = paragraph.add_run()
                tmp = q_index[q_index["content"]==paragraph.text]["area"].values[0]
                run.text = " ({0}) ".format(tmp)
                top_domain.append(tmp)
                font = run.font
                font.bold = True
            #Adjust text formatting
            for y in range(0,len(table.columns)):
                table.cell(0,y).vertical_anchor = MSO_ANCHOR.MIDDLE
                paragraph = table.cell(0,y).text_frame.paragraphs[0]
                paragraph.font.size = Pt(12)
                if y > 0:
                        paragraph.alignment = PP_ALIGN.CENTER
                else: 
                    continue
            for x in range(1,len(table.rows)):
                for y in range(0,len(table.columns)):
                    table.cell(x,y).vertical_anchor = MSO_ANCHOR.MIDDLE
                    paragraph = table.cell(x,y).text_frame.paragraphs[0]
                    paragraph.font.size = Pt(10.5)
                    if y > 0:
                        paragraph.alignment = PP_ALIGN.CENTER
                    else: 
                        continue
            #Adjust height of rows
            table.rows[0].height = Inches(0.50)
            for x in range(1,len(table.rows)):
                table.rows[x].height = Inches(0.1)
            table.columns[0].width = Inches(6.33)
            table.columns[1].width = Inches(1.5)
            table.columns[2].width = Inches(0.67)

            #Add Bottom Table
            top, left, width, height = Inches(4.30), Inches(0.75), Inches(8.5), Inches(2.33)
            shp = df_to_table(slide, bottom_chart, left, top, width, height)
            table = shp.table
            cell = table.cell(0,0)
            cell.text = "Bottom 5 Correlated Items"
            #track items in each domain for overview slide
            bottom_domain = []
            for x in range(1,len(table.rows)):
                paragraph = table.cell(x,0).text_frame.paragraphs[0]
                run = paragraph.add_run()
                tmp = q_index[q_index["content"]==paragraph.text]["area"].values[0]
                run.text = " ({0}) ".format(tmp)
                bottom_domain.append(tmp)
                font = run.font
                font.bold = True
            #Adjust text formatting
            for y in range(0,len(table.columns)):
                table.cell(0,y).vertical_anchor = MSO_ANCHOR.MIDDLE
                paragraph = table.cell(0,y).text_frame.paragraphs[0]
                paragraph.font.size = Pt(12)
                if y > 0:
                        paragraph.alignment = PP_ALIGN.CENTER
                else: 
                    continue
            for x in range(1,len(table.rows)):
                for y in range(0,len(table.columns)):
                    table.cell(x,y).vertical_anchor = MSO_ANCHOR.MIDDLE
                    paragraph = table.cell(x,y).text_frame.paragraphs[0]
                    paragraph.font.size = Pt(10.5)
                    if y > 0:
                        paragraph.alignment = PP_ALIGN.CENTER
                    else: 
                        continue
            #Adjust height of rows
            table.rows[0].height = Inches(0.50)
            for x in range(1,len(table.rows)):
                table.rows[x].height = Inches(0.1)
            table.columns[0].width = Inches(6.33)
            table.columns[1].width = Inches(1.5)
            table.columns[2].width = Inches(0.67)
            
            #collect data for overview slide
            overview[q] = [bottom_value, top_value, top_item, bottom_item, top_domain, bottom_domain]
    
        #Add Overview slide
        layout = prs.slide_layouts[7]
        slide = prs.slides.add_slide(layout)
        title = slide.shapes.title
        title.text = "{0} {1} {2} Overview".format(report_year, report_group, group_name)

        placeholder = slide.placeholders[10]
        paragraph = placeholder.text_frame
        run = paragraph.paragraphs[0].add_run()
        run.text = " "
        run.font.size = Pt(5)
        #For each overal question add overview text
        for q in overall_q:
            run = paragraph.add_paragraph()
            if report_group=='Non-Registered Family':
                run.text = "Overall Experience Q17"
            else:
                run.text = "Overall Experience {0}:".format(q)
            run.font.size = Pt(16)
            run.font.bold = True
            run = paragraph.add_paragraph()
            run.text = "The correlation coefficients (r) ranged from {0} to {1}.".format(overview[q][0],overview[q][1])
            run.font.size = Pt(12)
            run = paragraph.add_paragraph()
            if report_group=='Non-Registered Family':
                run.text = "The highest correlated item being {0} and the lowest correlated item being {1}.".format(q_index[q_index['question_no']==overview[q][2]]['non_reg_no'].values[0],q_index[q_index['question_no']==overview[q][3]]['non_reg_no'].values[0])
            else:
                run.text = "The highest correlated item being {0} and the lowest correlated item being {1}.".format(overview[q][2],overview[q][3])
            run.font.size = Pt(12)
            run = paragraph.add_paragraph()
            count_domain = [[x,overview[q][4].count(x)] for x in set(overview[q][4])]
            domain_value = []
            for x in count_domain:
                domain_value.append(x[0] + " (" + str(x[1])+")")
            domain_text = ', '.join(domain_value)
            run.text = "The domains captured in the top 5 correlated items included: {0}".format(domain_text)
            run.font.size = Pt(12)
            run = paragraph.add_paragraph()
            count_domain = [[x,overview[q][5].count(x)] for x in set(overview[q][5])]
            domain_value = []
            for x in count_domain:
                domain_value.append(x[0] + " (" + str(x[1])+")")
            domain_text = ', '.join(domain_value)
            run.text = "The domains captured in the bottom 5 correlated items included: {0}".format(domain_text)
            run.font.size = Pt(12)
            run.space_after = Pt(12)
        
        #Add Conclusion Slides
        #collect conclusion data
        conclusion_corr['item'] = q_value
        conclusion_corr['type'] = type_value
        conclusion_corr['rank_corr'] = ranking
        conclusion_corr['q_no'] = q_no
        #Add Conclusion summary
        if report_group == 'Non-Registered Family':
            layout = prs.slide_layouts[16]
        else:
            layout = prs.slide_layouts[4]
        slide = prs.slides.add_slide(layout)
        title = slide.shapes.title
        title.text = "Conclusions - Tying it All Together ({0})".format(report_group)
        
        #add conclusion slide
        if report_group == 'Non-Registered Family':
            layout = prs.slide_layouts[14]
            slide = prs.slides.add_slide(layout)
            title = slide.shapes.title
            title.text = "{0} Conclusions ({1})".format(report_year, report_group)
            #merge top/bottom data with correlation data    
            df_merge = pd.merge(conclusion, conclusion_corr, on=['item', 'type'])
            #Add question to correct quadrant
            placeholder = slide.placeholders[12]
            paragraph = placeholder.text_frame
            run = paragraph.paragraphs[0].add_run()
            run.text = 'Q17'
            run.font.bold = True
            run.font.underline = True
            run = paragraph.add_paragraph()
            run = paragraph.add_paragraph()
            df_filtered = df_merge[df_merge['type']==i][(df_merge['q_no']==overall_q.values[0]) & (df_merge['rank_tb']=='top') & (df_merge['rank_corr']== 'top')]
            non_reg_no = []
            for p in df_filtered['item'].values:
                non_reg_no.append(q_index[q_index['question_no']==p]['non_reg_no'].values[0])
            item_no = ', '.join(non_reg_no)
            
            if item_no == "":
                #if no question input N/A
                run.text = "N/A"
            else:
                run.text = item_no
            placeholder.text_frame.vertical_anchor = MSO_ANCHOR.MIDDLE
            
            placeholder = slide.placeholders[13]
            paragraph = placeholder.text_frame
            run = paragraph.paragraphs[0].add_run()
            run.text = 'Q17'
            run.font.bold = True
            run.font.underline = True
            run = paragraph.add_paragraph()
            run = paragraph.add_paragraph()
            df_filtered = df_merge[df_merge['type']==i][(df_merge['q_no']==overall_q.values[0]) & (df_merge['rank_tb']=='bottom') & (df_merge['rank_corr']== 'top')]
            non_reg_no = []
            for p in df_filtered['item'].values:
                non_reg_no.append(q_index[q_index['question_no']==p]['non_reg_no'].values[0])
            item_no = ', '.join(non_reg_no)
            if item_no == "":
                #if no question input N/A
                run.text = "N/A"
            else:
                run.text = item_no
            placeholder.text_frame.vertical_anchor = MSO_ANCHOR.MIDDLE
            placeholder = slide.placeholders[18]
            paragraph = placeholder.text_frame
            run = paragraph.paragraphs[0].add_run()
            run.text = 'Q17'
            run.font.bold = True
            run.font.underline = True
            run = paragraph.add_paragraph()
            run = paragraph.add_paragraph()
            df_filtered = df_merge[df_merge['type']==i][(df_merge['q_no']==overall_q.values[0]) & (df_merge['rank_tb']=='bottom') & (df_merge['rank_corr']== 'top')]
            non_reg_no = []
            for p in df_filtered['item'].values:
                non_reg_no.append(q_index[q_index['question_no']==p]['non_reg_no'].values[0])
            item_no = ', '.join(non_reg_no)
            if item_no == "":
                #if no question input N/A
                run.text = "N/A"
            else:
                run.text = item_no
            placeholder.text_frame.vertical_anchor = MSO_ANCHOR.MIDDLE
            placeholder = slide.placeholders[19]
            paragraph = placeholder.text_frame
            run = paragraph.paragraphs[0].add_run()
            run.text = 'Q17'
            run.font.bold = True
            run.font.underline = True
            run = paragraph.add_paragraph()
            run = paragraph.add_paragraph()
            df_filtered = df_merge[df_merge['type']==i][(df_merge['q_no']==overall_q.values[0]) & (df_merge['rank_tb']=='bottom') & (df_merge['rank_corr']== 'bottom')]
            non_reg_no = []
            for p in df_filtered['item'].values:
                non_reg_no.append(q_index[q_index['question_no']==p]['non_reg_no'].values[0])
            item_no = ', '.join(non_reg_no)
            if item_no == "":
                #if no question input N/A
                run.text = "N/A"
            else:
                run.text = item_no
            placeholder.text_frame.vertical_anchor = MSO_ANCHOR.MIDDLE
             
        else:
            layout = prs.slide_layouts[11]
            slide = prs.slides.add_slide(layout)
            title = slide.shapes.title
            title.text = "{0} Conclusions ({1}{2})".format(report_year, report_group, group_name)
            #merge top/bottom data with correlation data    
            df_merge = pd.merge(conclusion, conclusion_corr, on=['item', 'type'])
            #define each quadrant
            rank_table = [['top', 'top'], ['bottom', 'top'], ['top', 'bottom'], ['bottom', 'bottom']]
            #placeholder index starts at 10
            ph = 10
            #Add question to correct quadrant
            for no in range(0, len(rank_table)):
                rank_tb = rank_table[no][0]
                rank_corr = rank_table[no][1]
                for q in overall_q:
                    placeholder = slide.placeholders[ph]
                    paragraph = placeholder.text_frame
                    ph += 1
                    run = paragraph.paragraphs[0].add_run()
                    run.text = q
                    run.font.bold = True
                    run.font.underline = True
                    run = paragraph.add_paragraph()
                    run = paragraph.add_paragraph()
                    df_filtered = df_merge[df_merge['type']==i][(df_merge['q_no']==q) & (df_merge['rank_tb']==rank_tb) & (df_merge['rank_corr']== rank_corr)]
                    item_no = ', '.join(df_filtered['item'].values)
                    if item_no == "":
                        #if no question input N/A
                        run.text = "N/A"
                    else:
                        run.text = item_no
                    placeholder.text_frame.vertical_anchor = MSO_ANCHOR.MIDDLE

        #final Conclusions slide
        layout = prs.slide_layouts[5]
        slide = prs.slides.add_slide(layout)
        title = slide.shapes.title
        title.text = "{0} Final Conclusions ({1}{2})".format(report_year, report_group, group_name)
        x, y, cx, cy = Inches(0.67), Inches(1.48), Inches(8.58), Inches(2.27)
        shape = slide.shapes.add_table(2, 1, x, y, cx, cy)
        title_cell = shape.table.cell(0,0)
        title_cell.text = "{0} Highest Quality Improvement Area Focus ({1}{2})".format(report_year, report_group, group_name)
        title_cell = title_cell.text_frame.paragraphs[0]
        title_cell.font.size = Pt(16)
        title_cell.alignment = PP_ALIGN.CENTER
        shape.table.rows[0].height = Inches(0.4)
        cell = shape.table.cell(1,0)
        cell.margin_left = Inches(1)
        cell.margin_right = Inches(1)
        cell.margin_bottom = Inches(0.1)
        paragraph = cell.text_frame
        focus_area = df_filtered = df_merge[df_merge['type']==i][(df_merge['rank_tb']=='bottom') & (df_merge['rank_corr']=='top')]
        for q in set(focus_area['item']):
            run = paragraph.add_paragraph()
            item_content = q_index[q_index["question_no"]==q]["content"].values[0]
            domain = q_index[q_index["question_no"]==q]["area"].values[0]
            run.text = "Item {0} ({1})".format(item_content, domain)
            run.space_after = Pt(12)
        paragraph = cell.text_frame.paragraphs
        for x in paragraph:
            x.font.size = Pt(12)
        x, y, cx, cy = Inches(0.67), Inches(4.17), Inches(8.58), Inches(2.27)
        shape = slide.shapes.add_table(2, 2, x, y, cx, cy)
        table = shape.table
        table.columns[0].width = Inches(1.50)
        table.columns[1].width = Inches(7.09)
        title_cell = shape.table.cell(0,0)
        other_cell = table.cell(0,1)
        title_cell.merge(other_cell)
        title_cell.text = "Previous {0}{1} Highest Quality Improvement Area Focus".format(report_group, group_name)
        title_cell = title_cell.text_frame.paragraphs[0]
        title_cell.font.size = Pt(16)
        title_cell.alignment = PP_ALIGN.CENTER
        shape.table.rows[0].height = Inches(0.4)
        shape.table.cell(0,0).fill.solid()
        table.cell(0,0).fill.fore_color.rgb = RGBColor(182,162,194)
    return prs

In [ ]:
#---INPUT REQUIRED---#

#Load files
#Input report variables
report_month = input("Enter current month: ")
report_year = input("Enter report year: ")

#OPOC data
opoc_data = pd.read_csv("data/"+report_year+"_opoc_raw_data_cleaned.csv")

#Load OPOC Question dictionary file
wb = pd.ExcelFile('question_index.xlsx')

#Powerpoint layout template
layout = 'layout_template.pptx'

#Report Groups with Inpatient Numbers below n=50 only output Outpatient portion of report

groups = ['Non-Registered Family','Registered Client and Registered Family', 'Registered Family', 'Registered Client', "Acute Care", "CYEAP","CCR"]

for report_group in groups:
    #OPOC Questions not including overall category
    q_index = wb.parse('question_index')
    #OPOC Questions in overall category
    q_overall = wb.parse('overall')
    output_filename = "draft_ppt\draft_{0} {1}.pptx".format(report_year, report_group)
    if report_group == 'Non-Registered Family':
        #OPOC Questions for Non-Registered Family
        q_index = wb.parse('non_reg')
        q_overall = wb.parse('non_reg_ov')
        group_data = opoc_data[opoc_data['Registered Client']=="N"]
        summary_data = group_data
    if report_group == 'Registered Client and Registered Family':
        group_data = opoc_data[opoc_data['Registered Client']=="Y"]
        summary_data = group_data
    if report_group == 'Registered Family':
        group_data = opoc_data[(opoc_data['Registered Client']=="Y")&(opoc_data['Client Type']=="FAM")]
        summary_data = opoc_data[opoc_data['Registered Client']=="Y"]
    if report_group == 'Registered Client':
        group_data = opoc_data[(opoc_data['Registered Client']=="Y")&(opoc_data['Client Type']=="MH")]
        summary_data = opoc_data[opoc_data['Registered Client']=="Y"]
    if report_group == "Acute Care":
        group_data = opoc_data[(opoc_data['Registered Client']=="Y")&(opoc_data['clinical_program']=="Acute")]
        summary_data = group_data
    if report_group == "CYEAP":
        group_data = opoc_data[(opoc_data['Registered Client']=="Y")&(opoc_data['clinical_program']=="CYEAP")]
        summary_data = group_data
    if report_group == "CCR":
        group_data = opoc_data[(opoc_data['Registered Client']=="Y")&(opoc_data['clinical_program']=="CCR")]
        summary_data = group_data
    print(report_group)
    prs = Presentation(layout)
    prs, group = add_summary_slides(group_data, summary_data, prs, report_month, report_year, report_group)
    prs = add_demographic_slides(report_group, group_data, prs, group)
    prs, conclusion = add_tb_slides(group_data, prs, report_year, report_group, q_index, group)
    if report_group == 'Non-Registered Family':
        prs = add_nonreg_overall_slides(group_data, prs, report_year, report_group, q_index, q_overall, group)
    else:
        prs = add_overall_slides(group_data, prs, report_year, report_group, q_index, q_overall, group)
    prs = add_corr_slides(group_data, prs, report_year, report_group, q_index, q_overall, conclusion, group)
    prs.save(output_filename)


In [ ]:
#Top 10 Acute Program Breakdown 
opoc_data[opoc_data['clinical_program']=="Acute"]['Program'].value_counts().head(10)

In [ ]:
#Acute Programs - Use specific names under 'Program' 
groups = ["COMPASS", "Mood and Anxiety Ambulatory Services (MAAS)"]

for report_group in groups:
    #OPOC Questions not including overall category
    q_index = wb.parse('question_index')
    #OPOC Questions in overall category
    q_overall = wb.parse('overall')
    group_data = opoc_data[(opoc_data['Registered Client']=="Y")&(opoc_data['clinical_program']=="Acute")&(opoc_data['Program']==report_group)]
    summary_data = group_data
    report_group = input("Rename {0} as: ".format(report_group))
    output_filename = "draft_ppt\draft_{0} {1}.pptx".format(report_year, report_group)
    print(report_group)
    prs = Presentation(layout)
    prs, group = add_summary_slides(group_data, summary_data, prs, report_month, report_year, report_group)
    prs = add_demographic_slides(report_group, group_data, prs, group)
    prs, conclusion = add_tb_slides(group_data, prs, report_year, report_group, q_index, group)
    prs = add_overall_slides(group_data, prs, report_year, report_group, q_index, q_overall, group)
    prs = add_corr_slides(group_data, prs, report_year, report_group, q_index, q_overall, conclusion, group)
    prs.save(output_filename)